#  对于PDO_eConvs的简易复现

# 导入包

In [1]:
import torch
import numpy as np
from torch import nn
from torch.utils import data
from torch.utils.data import TensorDataset

import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import time

# 导入数据集 Rotated Mnist

In [2]:
def load_rotated_mnist(batch_size):
    #导入数据，将.amat格式转换为numpy array
    data_train = np.loadtxt('C:\\Users\\roderickzzc\\Desktop\\project\\pdo-ecov\\mnist_rotation_new\\mnist_all_rotation_normalized_float_train_valid.amat')
    data_test = np.loadtxt('C:\\Users\\roderickzzc\\Desktop\\project\\pdo-ecov\\mnist_rotation_new\\mnist_all_rotation_normalized_float_test.amat')

    # get train image datas
    x_train_val = data_train[:, :-1] / 1.0
    #由于原始数据集默认为784*1，现改为28*28
    x_train_val=np.reshape(x_train_val,(12000,1,28,28))
    x_test = data_test[:, :-1] / 1.0
    x_test=np.reshape(x_test,(50000,1,28,28))
    # get train image labels
    y_train_val = data_train[:, -1:]
    y_test = data_test[:, -1:]
    print(x_train_val[0].shape)
    
    # pytorch data loader
    #根据论文抽取2000个样本from training set作为validation
    train_val = torch.utils.data.TensorDataset(torch.Tensor(x_train_val), torch.Tensor(y_train_val))
    train, val = torch.utils.data.random_split(train_val, [10000,2000])
    test = torch.utils.data.TensorDataset(torch.Tensor(x_test), torch.Tensor(y_test))
    ## feature, label = train[0]
    ## print(feature.shape, label) 
    train_iter = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    val_iter = torch.utils.data.DataLoader(val, batch_size=batch_size, shuffle=True)
    test_iter = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
    


    return train_iter, val_iter, test_iter

In [3]:
batch_size=128
train_iter, val_iter, test_iter = load_rotated_mnist(batch_size)



(1, 28, 28)


# Generate a subgroup of O(2)

In [4]:
#定义所有的differential operator的filter
#这里是未经*1/h,1/(h^2),1/(h^3),1/(h^4)的版本
from sympy import *
d_0=torch.tensor([[0,0,0,0,0], [0,0,0,0,0], [0,0,1,0,0],[0,0,0,0,0],[0,0,0,0,0]])
d_x=Matrix([[0,0,0,0,0], [0,0,0,0,0], [0,-1/2,0,1/2,0],[0,0,0,0,0],[0,0,0,0,0]])
d_y=Matrix([ [0,0,0,0,0],[0,0,1/2,0,0], [0,0,0,0,0],[0,0,-1/2,0,0],[0,0,0,0,0]])
d_xx=Matrix([ [0,0,0,0,0],[0,0,0,0,0], [0,1,-2,1,0],[0,0,0,0,0],[0,0,0,0,0]])
d_xy=Matrix([ [0,0,0,0,0],[0,-1/4,0,1/4,0], [0,0,0,0,0],[0,1/4,0,-1/4,0],[0,0,0,0,0]])
d_yy=Matrix([ [0,0,0,0,0],[0,0,1,0,0], [0,0,-2,0,0],[0,0,1,0,0],[0,0,0,0,0]])
d_xxy=Matrix([[0,0,0,0,0],[0,1/2,-1,1/2,0], [0,0,0,0,0],[0,-1/2,1,-1/2,0],[0,0,0,0,0]])
d_xyy=Matrix([[0,0,0,0,0],[0,-1/2,0,1/2,0], [0,1,0,-1,0],[0,-1/2,0,1/2,0],[0,0,0,0,0]])
d_xxyy=Matrix([[0,0,0,0,0],[0,1,-2,1,0], [0,-2,4,-2,0],[0,1,-2,1,0],[0,0,0,0,0]])
d_xxx=Matrix([[0,0,0,0,0],[0,0,0,0,0], [-1/2,1,0,-1,1/2],[0,0,0,0,0],[0,0,0,0,0]])
d_yyy=Matrix([[0,0,1/2,0,0],[0,0,-1,0,0], [0,0,0,0,0],[0,0,1,0,0],[0,0,-1/2,0,0]])
d_xxxx=Matrix([[0,0,0,0,0],[0,0,0,0,0], [1,-4,6,-4,1],[0,0,0,0,0],[0,0,0,0,0]])
d_xxxy=Matrix([[0,0,0,0,0],[-1/4,1/2,0,-1/2,1/4], [0,0,0,0,0],[1/4,-1/2,0,1/2,-1/4],[0,0,0,0,0]])
d_xyyy=Matrix([[0,-1/4,0,1/4,0],[0,1/2,0,-1/2,0], [0,0,0,0,0],[0,-1/2,0,1/2,0],[0,1/4,0,-1/4,0]])
d_yyyy=Matrix([[0,0,1,0,0],[0,0,-4,0,0], [0,0,6,0,0],[0,0,-4,0,0],[0,0,1,0,0]])


In [5]:
#定义将pdo代入polynomail的函数
u,v=symbols("u v")
def subs_pdo(P, *args):

    P=expand(P)
  
    res=P.subs([(v**4,d_yyyy),(u*v**3,d_xyyy),(u**3*v,d_xxxy),(u**4,d_xxxx),(v**3,d_yyy),(u**3,d_xxx),\
                 (u**2*v**2,d_xxyy),(u*v**2,d_xyy),(u**2*v,d_xxy),(v**2,d_yy),(u*v,d_xy),(u**2,d_xx),(v,d_y),(u,d_x)])        
    res=np.array(res).astype(np.float64)
    return torch.tensor(res)

In [6]:
#测试
P=u
print(subs_pdo(P))
d_x

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -0.5000,  0.0000,  0.5000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]], dtype=torch.float64)


Matrix([
[0,    0, 0,   0, 0],
[0,    0, 0,   0, 0],
[0, -0.5, 0, 0.5, 0],
[0,    0, 0,   0, 0],
[0,    0, 0,   0, 0]])

In [7]:
#定义旋转函数，这个函数将act on我们的partial differential operators
def rotation(n):
    I=[]
    
    for k in range(0,n):
        
        theta=(2*k*pi/n)
        c, s = cos(theta), sin(theta)
        a = c*u - s*v
        b = s*u + c*v


        basis=[d_0,subs_pdo(a),subs_pdo(b),subs_pdo(expand(a*a)),subs_pdo(expand(a*b)),subs_pdo(expand(b*b)),subs_pdo(expand(a*a*b)),\
               subs_pdo(expand(a*b*b)),subs_pdo(expand(a*a*b*b))]
        basis=torch.stack(basis)
        I.append(basis)
        
    return torch.stack(I)
       
       

    

# 定义PDO_conv2D的Layer

In [8]:
def compute_linear_combination(pdo,beta,output_rot_num):
    L=[]
    for i in range(output_rot_num):
        #beta.size:1*9, pdo[i].size: 9*5*5, kernel_size=1*5*5
        term = beta[:, :,None, None] * pdo[i]
        kernel=torch.sum(term,1)
        L.append(kernel)#L_size: 8*1*5*5
            
    a=torch.stack(L,1)#a_size: 1*8*5*5
    return a

In [9]:
from torch.nn import Parameter
import torch.nn.functional as F
import math
from torch.nn.modules.utils import _pair


class PDO_Conv2D(nn.Module):

    def __init__(self, in_channels, out_channels, input_rot_num=1, output_rot_num=8):
        super().__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.input_rot_num = input_rot_num
        self.output_rot_num = output_rot_num
        #定义beta
        self.betas = nn.Parameter(torch.randn(self.in_channels*self.out_channels*self.input_rot_num,1,9))
        #nn.ParameterList([nn.Parameter(torch.randn(1,9)) for i in range(out_channels*in_channels*input_rot_num)] )#betas: (56*7)*1*9
        ##暂时未用kaiming_initialization
        self.reset_parameters()
        

           
    def reset_parameters(self):
        torch.nn.init.kaiming_uniform_(self.betas, a=math.sqrt(5),nonlinearity='relu')
        
        
   

    
    def forward(self, input):
        
        pdo=rotation(self.output_rot_num) #pdo: 8*9*5*5
        pdo=pdo.cuda()
        
        betas=self.betas.view(self.in_channels*self.input_rot_num,self.out_channels,1,9)
        
        U=[]
        for i in range(self.in_channels*self.input_rot_num): 
            in_kernel=torch.cat([compute_linear_combination(pdo, beta, self.output_rot_num) for beta in betas[i]]) #betas[i]_size:7*1*9 in_kernel: 7,8,5,5
            U.append(in_kernel)
        real_kernel=torch.stack(U) #1*7*8*5*5 & 56*7*8*5*5

             
        shape = (self.out_channels * self.output_rot_num,
                    self.in_channels * self.input_rot_num,    #56*1*5*5 & 56*56*5*5
                    5, 5)
        real_kernel = real_kernel.view(shape)
        real_kernel=real_kernel.float()
        

        input_shape = input.size()
        #input_size: 128,1,h,w & 128,56,h,w
        input = input.view(input_shape[0], self.in_channels*self.input_rot_num, input_shape[-2], input_shape[-1])
        #y_size: 128,56,h,w
        y = F.conv2d(input, weight=real_kernel, bias=None, stride=1,
                        padding=1)
        
        batch_size, _, ny_out, nx_out = y.size()
        y = y.view(batch_size, self.out_channels*self.output_rot_num, ny_out, nx_out)
        #y_size: 128,7,8,h,w

        return y

In [10]:
class P8_PDO_Conv_Z2(PDO_Conv2D):

    def __init__(self, *args, **kwargs):
        super(P8_PDO_Conv_Z2, self).__init__(in_channels=1, out_channels=7,input_rot_num=1, output_rot_num=8)


class P8_PDO_Conv_P8(PDO_Conv2D):

    def __init__(self, *args, **kwargs):
        super(P8_PDO_Conv_P8, self).__init__(in_channels=7, out_channels=7,input_rot_num=8, output_rot_num=8)


# 定义PDO_eConvs神经网络

In [11]:
class PDO_eConvs(nn.Module):
    def __init__(self):
        super(PDO_eConvs, self).__init__()
        self.conv1 = P8_PDO_Conv_Z2(1, 7,1,8)
        self.conv2 = P8_PDO_Conv_P8(7, 7,8,8)
        self.conv3 = P8_PDO_Conv_P8(7, 7,8,8)
        self.conv4 = P8_PDO_Conv_P8(7, 7,8,8)
        self.conv5 = P8_PDO_Conv_P8(7, 7,8,8)
        self.conv6 = P8_PDO_Conv_P8(7, 7,8,8)
        self.dropout=nn.Dropout(p=0.2)
        self.bn2 = nn.BatchNorm2d(56)
        self.maxpool2=nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(4*4*7*8, 50)
        self.fc2 = nn.Linear(50, 10)

        
    def forward(self, x):
        x = self.dropout(self.bn2(F.relu(self.conv1(x))))
        x = self.dropout(self.bn2(F.relu(self.conv2(x))))
        x = self.maxpool2(x)
        x = self.dropout(self.bn2(F.relu(self.conv3(x))))
        x = self.dropout(self.bn2(F.relu(self.conv4(x))))
        x = self.dropout(self.bn2(F.relu(self.conv5(x))))
        x = self.dropout(self.bn2(F.relu(self.conv6(x))))
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        y=torch.nn.functional.log_softmax(x)

        
        return y


In [12]:
maxpool2=nn.MaxPool2d(kernel_size=2, stride=2)
a=maxpool2(torch.randn(1,1,4,4))
a.size()

torch.Size([1, 1, 2, 2])

In [13]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)


In [14]:
net=PDO_eConvs()
net.apply(init_weights)
print(net)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = net.cuda()


PDO_eConvs(
  (conv1): P8_PDO_Conv_Z2()
  (conv2): P8_PDO_Conv_P8()
  (conv3): P8_PDO_Conv_P8()
  (conv4): P8_PDO_Conv_P8()
  (conv5): P8_PDO_Conv_P8()
  (conv6): P8_PDO_Conv_P8()
  (dropout): Dropout(p=0.2, inplace=False)
  (bn2): BatchNorm2d(56, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=896, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


<ipython-input-13-659b75651d94>:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


In [15]:
def evaluate_accuracy(data_iter, net, device=None):
    if device is None and isinstance(net, torch.nn.Module):
        # 如果没指定device就使用net的device
        device = list(net.parameters())[0].device
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(net, torch.nn.Module):
                net.eval() # 评估模式, 这会关闭dropout
                acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
                net.train() # 改回训练模式
            else: 
                if('is_training' in net.__code__.co_varnames): # 如果有is_training这个参数
                    # 将is_training设置成False
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() 
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
            n += y.shape[0]
    return acc_sum / n


# 训练

In [16]:
lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


In [17]:
def train(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs):
    print("training on ", device)
    loss = torch.nn.NLLLoss()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, batch_count, start = 0.0, 0.0, 0, 0, time.time()
        for X, y in train_iter:
            X = X.to(device)
            y=y.type(torch.LongTensor)
            y = y.to(device)
            y=y.view(1,-1)[0]
                   
            y_hat = net(X)
            
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, test_acc, time.time() - start))
        
        if epoch==(0.5*num_epochs or 0.75*num_epochs):
            lr=0.1*lr


In [ ]:
train(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda


<ipython-input-11-5f4db8ede61c>:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  y=torch.nn.functional.log_softmax(x)


epoch 1, loss 2.5968, train acc 0.118, test acc 14.501, time 935.0 sec
